# Language Modeling using Ngram

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import shutil
shutil.copy("/content/drive/MyDrive/FRA 501 IntroNLP&DL/Dataset/BEST2010.zip", "/content/BEST2010.zip")

'/content/BEST2010.zip'

In [3]:
!unzip BEST2010.zip

Archive:  BEST2010.zip
   creating: BEST2010/
  inflating: BEST2010/article.txt    
  inflating: BEST2010/encyclopedia.txt  
  inflating: BEST2010/news.txt       


In [18]:
#First we import necessary library such as math, nltk, bigram, and collections.
import math
import nltk
import io
import random
from random import shuffle
from collections import defaultdict
random.seed(999)

BEST2010 is a free Thai NLP dataset by NECTEC usually use as a standard benchmark for various NLP tasks includeing language modeling. BEST2010 is separated into 4 domain article, encyclopedia, news and novel. The data is already  tokenized using '|' as a separator.

For example,

ตาม|ที่|นางประนอม ทองจันทร์| |กับ| |ด.ช.กิตติพงษ์ แหลมผักแว่น| |และ| |ด.ญ.กาญจนา กรองแก้ว| |ป่วย|สงสัย|ติด|เชื้อ|ไข้|ขณะ|นี้|ยัง|ไม่|ดี|ขึ้น|

In [5]:
# We choose news domain as our dataset
best2010=[]
fp= io.open('BEST2010/news.txt','r',encoding='utf-8')
for i,line in enumerate(fp):
    best2010.append(line.strip()[:-1])
fp.close()
all_vocabulary =set()
total_word_count =0
for line in best2010:
    for word in line.split('|'):        
        all_vocabulary.add(word)
        total_word_count+=1

In [7]:
best2010[0]

'สงสัย|ติด|หวัด|นก| |อีก|คน|ยัง|น่า|ห่วง'

In [8]:
best2010[1]

'ตาม|ที่|นางประนอม ทองจันทร์| |กับ| |ด.ช.กิตติพงษ์ แหลมผักแว่น| |และ| |ด.ญ.กาญจนา กรองแก้ว| |ป่วย|สงสัย|ติด|เชื้อ|ไข้|ขณะ|นี้|ยัง|ไม่|ดี|ขึ้น'

In [10]:
#For simplicity, we assumes that each line is a sentence.
print ('Total sentences in BEST2010 news dataset :\t'+ str(len(best2010)))
print ('Total word counts in BEST2010 news dataset :\t'+ str(total_word_count))
print ('Total vocabulary in BEST2010 news dataset :\t'+ str(len(all_vocabulary)))

Total sentences in BEST2010 news dataset :	30969
Total word counts in BEST2010 news dataset :	1660190
Total vocabulary in BEST2010 news dataset :	35488


We separate out input into 2 sets, train and test data with 70:30 ratio

In [11]:
sentences = best2010
# The data is separated to train and test set with 70:30 ratio.
train = sentences[:int(len(sentences)*0.7)]
test = sentences[int(len(sentences)*0.7):]

#Training data
train_vocabulary =set()
train_word_count =0
for line in train:
    for word in line.split('|'):        
        train_vocabulary.add(word)
        train_word_count+=1
print ('Total sentences in BEST2010 news training dataset :\t'+ str(len(train)))
print ('Total word counts in BEST2010 news training dataset :\t'+ str(train_word_count))
print ('Total vocabuary in BEST2010 news training dataset :\t'+ str(len(train_vocabulary)))
# We will use 1/vocab_size as a default value for unknown word
unk_value = math.pow(len(train_vocabulary),-1)

Total sentences in BEST2010 news training dataset :	21678
Total word counts in BEST2010 news training dataset :	1042797
Total vocabuary in BEST2010 news training dataset :	26240


In [45]:
print(unk_value)

3.8109756097560976e-05


In [16]:
train[:10]

['สงสัย|ติด|หวัด|นก| |อีก|คน|ยัง|น่า|ห่วง',
 'ตาม|ที่|นางประนอม ทองจันทร์| |กับ| |ด.ช.กิตติพงษ์ แหลมผักแว่น| |และ| |ด.ญ.กาญจนา กรองแก้ว| |ป่วย|สงสัย|ติด|เชื้อ|ไข้|ขณะ|นี้|ยัง|ไม่|ดี|ขึ้น',
 'หลัง|เข้า|เยี่ยม|ดู|อาการ|ผู้|ป่วย|แล้ว| |น.พ.จรัล|ประชุม|ร่วม|กับ|เจ้าหน้าที่|ทุก|ฝ่าย| |เพื่อ|สรุป|ผล|การ|ดำเนิน|การ| |รวม|ทั้ง|สอบสวน|โรค|ก่อน|ที่|ผู้|ป่วย|จะ|ถูก|ส่ง|มา|รักษา|ตัว| |จาก|นั้น|ร่วม|กัน|แถลง|ข่าว| |โดย| |น.พ.จรัล|กล่าว|ว่า| |ขณะ|นี้|ผู้|ป่วย|ทั้ง| |3| |ราย| |อาการ|ยัง|ทรง| |โดย|ใน|ราย|ของ| |ด.ช.กิตติพงษ์| |กับ| |ด.ญ.กาญจนา| |ปอด|หาย|เป็น|ปกติ|แล้ว| |คาด|ว่า|จะ|กลับ|บ้าน|ได้|ใน|ไม่|ช้า|นี้| |แต่|ใน|ราย|ของ|นางประนอม|อาการ|ยัง|น่า|เป็นห่วง| |ซึ่ง|ทั้ง| |3| |ราย| |ใน|ชั้น|นี้|ถือ|ว่า|เป็น|ผู้|ป่วย|อยู่|ใน|ขั้น|น่า|สงสัย|อาจ|ติด|เชื้อ|ไข้|หวัด|นก| |เพราะ|ตรวจ|พบ|ผู้|ป่วย|มี|อาการ|ปอด|บวม|ปอด|อักเสบ| |เนื่อง|จาก|ติด|เชื้อ|ไวรัส| |แต่|ยัง|สรุป|ไม่|ได้|ว่า|ติด|เชื้อ|ไข้|หวัด|นก|แน่ชัด|หรือ|ไม่| |ต้อง|รอ|ผล|ตรวจ|จาก|ห้อง|ปฏิบัติการ|ที่|ได้|ส่ง|ตัวอย่าง|เลือด| |ไป|ตรวจ|พิสูจน์|ที่|กรมวิทยาศ

In [12]:
train_word_count

1042797

In [47]:
train_vocabulary

{'',
 'ซอยมังกร',
 'ก้าวร้าว',
 'ประจำใจ',
 'นายรองพล เจริญพันธ์',
 'ต.ปากแคว',
 'สุก',
 'ซอยพหลโยธิน 52',
 'นายอภิสิทธิ์เวชชาชีวะ',
 'พ.ต.ต.ชาตรี',
 'พระโอรส',
 'กรมคุมประพฤติ',
 'โจษขาน',
 'ปราบปรามการกระทำผิดต่อเด็ก เยาวชน และสตรี',
 'บูฆอรี นิ้วหลี',
 'นายจิตรพงษ์ กว้างสุขสถิตย์',
 'ปวด',
 'สภอ.สตึก',
 'หม่อมหลวง',
 'ชอบใจ',
 'หน้าแตก',
 'กัมป์ ไอเนส',
 'เมืองพัทยา',
 'อุทยานแห่งชาติเขาใหญ่',
 'บ้านทุ่งยามู',
 'มูลฟ้อง',
 'ป.ป.ช./NE>',
 'นายวิเชียร โคตรเอื้อม',
 'พี่น้อง',
 'โรงพยาบาลสมเด็จพระสังฆราช',
 'บริษัทท่าอากาศยานไทยจำกัด(มหาชน',
 'แจ็กเก็ต',
 'รับผิดชอบ',
 'เข้มงวด',
 'นายพิจิตต รัตตกุล',
 '481',
 'นายสุทิน เจริญรักษา',
 'กระทรวงคมนาคม',
 '6438',
 'นายวิโรจน์ แก้วพิทยา',
 'ลาง',
 'สุ่ม',
 'บริษัท ไอทีวี จำกัด(มหาชน)',
 'ไอเอ็นเอ็น',
 'นายอุกฤษฎ์ถาวรไกรกุล',
 'วัดราษฏร์สโมสร',
 'นายเจษฎา จันทร์ดี',
 'พ.ต.อ.มาโนช',
 'ทรัพยากรธรรมชาติฯ',
 'รพ.ภูมิพลอดุลยเดช',
 'ก.พ.',
 'เว็บบอร์ด',
 'ลาดหลุมแก้ว',
 'ศูนย์ปฏิบัติการตำรวจส่วนหน้า',
 'กองวินัย',
 'นางเฉิง',
 'เซรุ่ม',
 'พล.ร.อ.ส

# Unigram

In this section, we will demonstrate how to build a unigram language model <br>
**Important note:** <br>
**\<s\>** = sentence start symbol <br>
**\</s\>** = sentence end symbol 

In [44]:
# lambda example
a = lambda x,y:x*y
print(a(10,2))

20


In [43]:
# defaultdict example
test_dict = defaultdict(lambda: 0)
test_dict["A"] = 1
test_dict["B"] = 2

print(test_dict["A"])
print(test_dict["A"])
print(test_dict["C"])

1
1
0


In [30]:
def getUnigramModel(data):
    model = defaultdict(lambda: 0)
    word_count =0
    for sentence in data:
        sentence +=  u'|</s>' #for unigram model we can always ignore <s>, since p(w0=<s>)=1
        for w1 in sentence.split('|'):
            model[w1] +=1.0
            word_count+=1
    for w1 in model:
        model[w1] = model[w1]/(word_count)
    return model

In [31]:
model = getUnigramModel(train)

In [32]:
model

defaultdict(<function __main__.getUnigramModel.<locals>.<lambda>()>,
            {'สงสัย': 0.00037952981516710117,
             'ติด': 0.000799455130463374,
             'หวัด': 0.0007026938162004744,
             'นก': 0.0007026938162004744,
             ' ': 0.12474694098029546,
             'อีก': 0.0023156955306606543,
             'คน': 0.006206815566359003,
             'ยัง': 0.0035905023603184667,
             'น่า': 0.0007985157002278118,
             'ห่วง': 8.266986072946758e-05,
             '</s>': 0.020364968646515887,
             'ตาม': 0.0019427417271424881,
             'ที่': 0.02245050376946382,
             'นางประนอม ทองจันทร์': 9.394302355621316e-07,
             'กับ': 0.004522417153996101,
             'ด.ช.กิตติพงษ์ แหลมผักแว่น': 9.394302355621316e-07,
             'และ': 0.01204443505014209,
             'ด.ญ.กาญจนา กรองแก้ว': 9.394302355621316e-07,
             'ป่วย': 0.00047535169919443856,
             'เชื้อ': 0.0007252401418539656,
             'ไข้': 0

In [ ]:
def getLnValue(x):
    if x >0.0:
        return math.log(x)
    else:
        return math.log(unk_value)

In [ ]:
#problability of 'นายก'
print(getLnValue(model[u'นายก']))
#for example, problability of 'นายกรัฐมนตรี' which is an unknown word is equal to
print(getLnValue(model[u'นายกรัฐมนตรี']))
#problability of 'นายก' 'ได้' 'ให้' 'สัมภาษณ์' 'กับ' 'สื่อ'
prob = getLnValue(model[u'นายก'])+getLnValue(model[u'ได้'])+ getLnValue(model[u'ให้'])+getLnValue(model[u'สัมภาษณ์'])+getLnValue(model[u'กับ'])+getLnValue(model[u'สื่อ'])+getLnValue(model['</s>'])
print ('Problability of a sentence', math.exp(prob))

-6.551526663995246
-10.175040243058024
Problability of a sentence 5.617210748667918e-18
